In [5]:
from random import randrange, uniform, randint, choice, choices
import random
import numpy as np
import math
import copy
from tabulate import tabulate
from num2words import num2words
from Examen_2 import random_poblation, random_probabilidades,population_sorted, operation_dot_product, PN, piscina_de_aparenamiento, operation_aptitud, mutate_population,recombination_method2,add_offspring_to_repairpopulation, RP,equal_lists 


## Final Version of the Genetic Algorithm for the Backpack Problem

In this final version, I attempt to solve the backpack problem using a genetic algorithm. I have organized my code into two files, with this code serving as the main script that calls various functions defined in `Examen_2.py`. The following functions from `Examen_2.py` are crucial for solving this problem:

- `random_poblation`
- `random_probabilidades`
- `population_sorted`
- `operation_dot_product`
- `PN`
- `piscina_de_aparenamiento`
- `operation_aptitud`
- `mutate_population`
- `recombination_method2`
- `add_offspring_to_repairpopulation`
- `RP`
- `equal_lists`

These functions are used to handle different aspects of the genetic algorithm for the backpack problem. By separating them into a dedicated module, the code is more organized and easier to manage. The main script in this file calls these functions to execute the genetic algorithm for various problem instances.


In [6]:
def weight_random_pairing(cromosomas, costos, x_rate):
    # We order from Greater to Minor in order descending
    # costos = sorted(costos, reverse=True)
    cromosomas, costos = population_sorted(cromosomas, costos)
    # selected_cromos = math.ceil(len(cromosomas) * x_rate)

    # selected_costs = math.ceil(len(costos) * x_rate)

    # selected_cromos = cromosomas[:selected_cromos]
    # selected_costs = costos[:selected_costs]

    # not_selected_cromos = cromosomas[selected_cromos:]
    # not_selected_costs = costos[selected_costs:]

    # min_not_selected_cost = min(not_selected_costs)

    selected_cromos = math.ceil(len(cromosomas) * x_rate)
    selected_costs = math.ceil(len(costos) * x_rate)

    selected_elements_cromos = cromosomas[:selected_cromos]
    selected_elem_costs = costos[:selected_costs]
    # selected_elem_costs = [23.86569316, 19.58156339, 17.52276708, 16.06364778]
    not_selected_costs = costos[selected_costs:]
    min_not_selected_costs = min(not_selected_costs)

    # cromosoma_descartado = cromosomas_dict[min_not_selected_costs]
    cn_list = []

    for ci in selected_elem_costs:
        temp_cn = ci - (min_not_selected_costs)
        cn_list.append(temp_cn)

    sumatory_cn_list = sum(cn_list)
    sum_valores_pn, valores_pn = PN(cn_list)
    numeros_aleatorios = random_probabilidades(len(valores_pn))

    padres = []
    madres = []
    append_to_madres = True
    for numero_aleatorio in numeros_aleatorios:
        for i, acumulado in enumerate(sum_valores_pn):
            if numero_aleatorio <= acumulado:
                if append_to_madres:
                    padres.append(selected_elements_cromos[i])
                else:
                    madres.append(selected_elements_cromos[i])
                append_to_madres = not append_to_madres
                break
            # else:
            #     lista_seleccionada = choice([padres, madres])
            #     lista_seleccionada.append(selected_elements[-1])

    # print("Pesos de los objetos: \n", pesos)
    # print("Valores de cada objeto")

    # print("\nCromosomas descartados: \n", not_selected_costs)
    # # print("Cromosoma descartado: " , min_not_selected_costs)
    # print("Cromsomas seleccionados: \n", selected_elem_costs)
    # print("Fitness del cromosoma descartado: ",min_not_selected_costs)
    # print("\nValores de Cn: \n", cn_list)
    # print("Valores de sumatoria de costos ", sumatory_cn_list)
    # print("\nValores de Pn :\n",valores_pn,"\nSumatoria de las probabilidades:\n",sum_valores_pn)
    # print("\nNumeros aleaotorios: \n", numeros_aleatorios)
    # print("madres: \n", madres)
    # print("padres: \n", padres)
    return padres, madres



def tournament_selection(poblacion, costos, x_rate):
    poblacion, costos = population_sorted(poblacion, costos)

    # selected_cromos = math.ceil(len(poblacion) * x_rate)
    selected_costs = math.ceil(len(costos) * x_rate)

    selected_elem_costs = costos[:selected_costs]

    lista_var_padres = piscina_de_aparenamiento(selected_elem_costs, costos)

    padres = []
    madres = []
    append_to_madres = True

    for valor in lista_var_padres:
        if append_to_madres:
            padres.append(poblacion[valor])
        else:
            madres.append(poblacion[valor])
        append_to_madres = not append_to_madres

    return padres, madres



Above the function are related with our selection methods one for weight random pairing and the other one for tournament selection, those were the most dificult part to separate them correctly related with the theory.

### Now I would explain how this GA works for knapsack problem

## Genetic Algorithm for Knapsack Problem

This code is an implementation of a genetic algorithm to solve the 0/1 Knapsack Problem. The algorithm is run for a specific problem instance, and the entire process is repeated for multiple runs.

### Variable Initialization

- `best_cromos_after_runs`, `worst_cromos_after_runs`, `best_fitnes_after_runs`, and `worst_fitness_after_runs` are lists that will store information about the best and worst chromosomes and their fitness after each run.
- `n_runs` is set to 30, indicating that the genetic algorithm will be executed 30 times.
- `n_generations` is set to 1000, specifying the number of generations or iterations in each run.
- `number_list` is created to store numbers from 1 to `n_runs`.

### Main Loop

The primary part of the code is a loop that runs the genetic algorithm `n_runs` times. In each run, the algorithm is executed with different random populations.

1. Inside the loop, various parameters and lists are initialized, including `population_size`, `objects`, `selection_rate`, `mutation_rate`, `sum_knapsack_capacity`, `pesos` (weights of items), and `valores` (values of items).

2. A list `lista` is initialized to store populations. This list starts with a population obtained from `random_poblation()` and is then repaired using `RP()` to ensure the knapsack constraint is satisfied. This repaired population is added to the list `lista`.

3. The code enters another loop representing the generations of the genetic algorithm. In each generation, the following steps are performed:
   - **Selection:** Tournament selection is used to select parents for reproduction, and the selected parents are stored in `select_padres` and `select_madres`.
   - **Fitness Calculation:** The fitness of the selected parents is calculated and stored in `lista_antes_de_aptitud`.
   - **Recombination:** A recombination method is applied to the selected parents, and the resulting offspring are stored in `recombinated_list`.
   - **Mutation:** The offspring in `recombinated_list` are mutated, and the mutated population is stored in `mutated_list`.
   - **Addition of Offspring to the Population:** The mutated offspring are added to the current population while maintaining a constant population size. The population is then repaired to satisfy the knapsack constraint, and the repaired population is stored in `mutate_repair_list`.
   - Finally, the old population is removed from the `lista`, and the new repaired population is added to it.

4. After the loop for generations, the code sorts the populations and stores the best and worst chromosomes and their fitness in the `temp_best_cromos` and `fitness_sorted` lists.

5. The best chromosome and fitness from the last generation of the run are appended to `best_cromos_after_runs` and `best_fitnes_after_runs`, while the worst chromosome and fitness are appended to `worst_cromos_after_runs` and `worst_fitness_after_runs`.

The code essentially runs a genetic algorithm to solve the knapsack problem for 30 different runs, each with its own random initial population. It collects statistics about the best and worst solutions found in each run. This code is likely part of an experiment to assess the performance and variability of the genetic algorithm for this specific problem instance.


In [7]:
# This 4 list are for my best chromosomes and worst chromosomes with their fitness.
best_cromos_after_runs = []
worst_cromos_after_runs = []
best_fitnes_after_runs = []
worst_fitness_after_runs = []

n_runs = 30
n_generations = 1000
number_list = list(range(1, n_runs + 1))

for _ in range(n_runs):
    # For our test this was our inputs for the backpack including all this things
    population_size = 500
    objects = 15
    selection_rate = 0.5
    mutation_rate = 0.2
    sum_knapsack_capacity = 750
    pesos = [70, 73, 77, 80, 82, 87, 90, 94, 98, 106, 110, 113, 115, 118, 120]
    valores = [135, 139, 149, 150, 156, 163, 173, 184, 192, 201, 210, 214, 221, 229, 240]
    lista = []

    temp_best_cromos = []
    temp_best_fitness = []
    resultado_random_poblacion = random_poblation(population_size, objects)
    poblacion_reparada = RP(resultado_random_poblacion, pesos, sum_knapsack_capacity)
    #Append the repair population to our list, to start our runs with this generations.
    lista.append(poblacion_reparada)

    for _ in range(n_generations):
        
        # I call our function for selection in this case tournament selection and saved it in select_padres, select_madres
        select_padres, select_madres = tournament_selection(
            lista[0], valores, selection_rate
        )

        lista_antes_de_aptitud = operation_aptitud(select_padres + select_madres, valores)
        

        # I call our function for recombination and saved it in recombinated_list

        recombinated_list = recombination_method2(
            select_padres, select_madres, selection_rate
        )

        # I call our function for recombination and saved it in mutated_list

        mutated_list = mutate_population(recombinated_list, mutation_rate)

        population_w_mutation_list = add_offspring_to_repairpopulation(
            poblacion_reparada, mutated_list, selection_rate, valores
        )
        # The next step is to repair our mutated children to be ready to add them correctly to our population 
        mutate_repair_list = RP(population_w_mutation_list, pesos, sum_knapsack_capacity)
        
        lista.pop(0)
        lista.append(mutate_repair_list)
        best_cromos_afterLoop, fitness_sorted = population_sorted(
            mutate_repair_list, valores
        )
        temp_best_cromos.append(best_cromos_afterLoop[0])


    x_cromos, y_fitness = population_sorted(temp_best_cromos, valores)
    x_cromos, y_fitness = population_sorted(temp_best_cromos, valores)
    best_cromos_after_runs.append(x_cromos[0])
    worst_cromos_after_runs.append(x_cromos[-1])
    best_fitnes_after_runs.append(y_fitness[0])
    worst_fitness_after_runs.append(y_fitness[-1])





The part below is related with how I created the tables with the best chromosomes and worst chromosomes to visualize better the outputs.

In [8]:
data = list(zip(number_list,best_cromos_after_runs,best_fitnes_after_runs,))

table = tabulate(data, headers=["Runs","Best Chromosome", "Best Fitness"], tablefmt="fancy_grid")

print(table)

data2 = list(zip(number_list,worst_cromos_after_runs,worst_fitness_after_runs))

table2 = tabulate(data2, headers=["Runs","Worst Chromosome","Worst Fitness"], tablefmt="fancy_grid")

print(table2)

╒════════╤═══════════════════════════════════════════════╤════════════════╕
│   Runs │ Best Chromosome                               │   Best Fitness │
╞════════╪═══════════════════════════════════════════════╪════════════════╡
│      1 │ [1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1] │           1458 │
╘════════╧═══════════════════════════════════════════════╧════════════════╛
╒════════╤═══════════════════════════════════════════════╤═════════════════╕
│   Runs │ Worst Chromosome                              │   Worst Fitness │
╞════════╪═══════════════════════════════════════════════╪═════════════════╡
│      1 │ [1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1] │            1450 │
╘════════╧═══════════════════════════════════════════════╧═════════════════╛
